# FEATURE ENGINEERING

## 1. IMPORTS

In [6]:
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay
from imblearn.under_sampling import RandomUnderSampler

from src.data_preprocessing import engineer_features, impute_missing_values
from src.outlier_detection import remove_outliers_iqr
from src.model_training import train_model
from src.monitoring import check_drift, check_model_degradation

## 2. CARGA DE DATOS

In [7]:
df = pd.read_csv("data/fraud.csv")

# 3. INGENIERIA DE VARIABLES

In [8]:
target = 'isFraud'

# Crear nuevas variables
df = engineer_features(df)

# Elegir columnas numéricas a imputar
num_cols = ['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
            'oldbalanceDest', 'newbalanceDest',
            'balance_diff_orig', 'balance_diff_dest', 'amount_to_balance_ratio']

## 3.1 IMPUTACION DE DATOS

La imputación de datos no se aplica adrede sino que requiere de un analisis del negocio concreto, la principal pregunta que ayuda a resolver que estrategia utilizar son:

Qué significa para el negocio un valor nulo en x variable?

Y de esa respuesta surgen las siguientes preguntas:

Puede ser rellenada como un valor faltante?
Puede ser rellenada con un valor calculado?
Podemos quitar ese dato? La respuesta a esta pregunta es mas interna y la respondemos en base a si es un dato de fraude=True por ejemplo en este caso es relevante y quizas no es una opción quitarla porque nos esta brindando informacion, y en este caso esa información cobra relevancia porque es de la clase minoritaria.

Y en base a eso decidimos que estrategia se adapta mejor mediante experimentación para el entrenamiento del modelo

In [9]:
# Imputar
X = df.drop(columns=['nameOrig', 'nameDest', 'isFraud', 'isFlaggedFraud'])  # sin IDs ni target
y = df['isFraud']
X = impute_missing_values(X, num_cols, save_path="models/imputer.pkl")

In [10]:
X.to_csv("data/fraud_dataset_encoded.csv", index=False)

# 4. OUTLIER DETECTION

In [11]:
# OUTLIER DETECTION
df_imputed = X.copy()
df_imputed[target] = y
df_filtered = remove_outliers_iqr(df_imputed, num_cols, target)

X = df_filtered.drop(columns=target)
y = df_filtered[target]


# 6. BALANCEO DE CLASES

In [12]:

print("Distribución original:", y.value_counts())
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)
print("Distribución balanceada:", y_resampled.value_counts())


Distribución original: isFraud
0    2131605
1       8213
Name: count, dtype: int64
Distribución balanceada: isFraud
0    8213
1    8213
Name: count, dtype: int64


/Users/juamp/miniconda3/envs/fraud_detection/lib/python3.9/site-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/Users/juamp/miniconda3/envs/fraud_detection/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


# 7. ENTRENAMIENTO

In [13]:

model, X_train, X_test, y_train, y_test = train_model(X_resampled, y_resampled, save_path="models/rf_model.pkl")

# 8. EVALUACIÓN

In [14]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd

model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

scoring = {
    "roc_auc": "roc_auc",
    "precision": make_scorer(precision_score),
    "recall": make_scorer(recall_score),
    "f1": make_scorer(f1_score)
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_results = cross_validate(model, X_resampled, y_resampled, cv=cv, scoring=scoring, return_train_score=False)

# Convertir a DataFrame
results_df = pd.DataFrame(cv_results)

# Mostrar métricas promedio
print("✅ Evaluación cruzada (5-fold):")
for metric in scoring.keys():
    scores = results_df[f"test_{metric}"]
    print(f"{metric.upper():<10}: {scores.mean():.4f} ± {scores.std():.4f}")

✅ Evaluación cruzada (5-fold):
ROC_AUC   : 1.0000 ± 0.0000
PRECISION : 0.9999 ± 0.0003
RECALL    : 0.9994 ± 0.0004
F1        : 0.9996 ± 0.0003


## ✅ Evaluación del Modelo con Validación Cruzada

Se entrenó y evaluó el modelo usando distintos grupos de datos para asegurarnos de que no solo funcione bien con un conjunto, sino que sea consistente en varios escenarios.

### 📊 Resultados Promedio (5 repeticiones):

| Métrica     | Resultado promedio | Qué significa |
|-------------|--------------------|----------------|
| **AUC**     | 1.0000 ± 0.0000     | El modelo distingue perfectamente entre transacciones normales y fraudulentas. |
| **Precisión** | 0.9999 ± 0.0003     | Cuando el modelo predice fraude, casi siempre acierta. Muy pocos falsos positivos. |
| **Recall**    | 0.9994 ± 0.0004     | Detecta prácticamente todos los fraudes reales. Muy pocos se escapan. |
| **F1 Score**  | 0.9996 ± 0.0003     | Excelente equilibrio entre precisión y cobertura de los fraudes. |

---

### ✅ Conclusión

El modelo demuestra un rendimiento **excepcional y muy estable**. A lo largo de múltiples pruebas, **detecta casi todos los fraudes sin generar muchas falsas alarmas**. Esto lo convierte en una solución muy confiable para implementación en producción.

# 9. MONITOREO - SIMULACIÓN

In [10]:

df_current = X_test.copy()
df_current['isFraud'] = y_test.values
baseline_auc = roc_auc_score(y_test, y_proba)

drift_results = check_drift(df_current, X_train, num_cols)
new_y_proba = model.predict_proba(df_current.drop(columns='isFraud'))[:, 1]
perf_results = check_model_degradation(df_current['isFraud'], new_y_proba, baseline_auc)

# 10. RESULTADOS MONITOREO

In [11]:
print("🎯 Resultados de Drift:")
for col, result in drift_results.items():
    print(f"- {col}: Drift Score = {result['drift_score']:.4f} | Drift Detectado: {result['drifted']}")

print("\n📉 Evaluación de Desempeño:")
print(f"- AUC actual: {perf_results['current_auc']:.4f}")
print(f"- Drop desde baseline: {perf_results['auc_drop']:.4f}")
print(f"- ¿Reentrenar? {'✅ SÍ' if perf_results['retrain'] else '❌ NO'}")

🎯 Resultados de Drift:
- step: Drift Score = 4.9717 | Drift Detectado: True
- amount: Drift Score = 63731.1390 | Drift Detectado: True
- oldbalanceOrg: Drift Score = 79221.4090 | Drift Detectado: True
- newbalanceOrig: Drift Score = 15534.7028 | Drift Detectado: True
- oldbalanceDest: Drift Score = 56711.4124 | Drift Detectado: True
- newbalanceDest: Drift Score = 94877.4804 | Drift Detectado: True
- balance_diff_orig: Drift Score = 64728.5591 | Drift Detectado: True
- balance_diff_dest: Drift Score = 47520.9335 | Drift Detectado: True
- amount_to_balance_ratio: Drift Score = 491.9837 | Drift Detectado: True

📉 Evaluación de Desempeño:
- AUC actual: 1.0000
- Drop desde baseline: 0.0000
- ¿Reentrenar? ❌ NO


# Armar archivo de monitoring_metrics.csv

In [12]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

np.random.seed(42)

# Simulamos 30 días
n_days = 30
base_date = datetime.today() - timedelta(days=n_days)
dates = [base_date + timedelta(days=i) for i in range(n_days)]

# Generamos precisión y recall simulados
precision = np.clip(np.random.normal(0.91, 0.03, n_days), 0.75, 0.99)
recall = np.clip(np.random.normal(0.90, 0.04, n_days), 0.70, 0.99)

# F1 Score (2 * P * R / (P + R))
f1 = 2 * precision * recall / (precision + recall)

data = {
    "date": dates,
    "auc": np.clip(np.random.normal(0.94, 0.02, n_days), 0.85, 0.99),
    "precision": precision,
    "recall": recall,
    "f1_score": f1,
    "drift_score": np.clip(np.random.normal(0.08, 0.04, n_days), 0.01, 0.25),
}

# Reentreno simulado
data["retrain_triggered"] = [
    int(data["auc"][i] < 0.88 or data["drift_score"][i] > 0.15)
    for i in range(n_days)
]

df = pd.DataFrame(data)
df.to_csv("data/monitoring_metrics.csv", index=False)

# Armar graficos de SHAP

In [ ]:
import shap
import matplotlib.pyplot as plt
import pandas as pd
import joblib

# Cargar modelo y datos
model = joblib.load("models/rf_model.pkl")

# Calcular SHAP values globales
explainer = shap.TreeExplainer(model)

# Extraer SHAP values solo para la clase 1 
shap_values_class_1 = shap_values[:, :, 1] 

# Bar plot
import numpy as np

# Media absoluta de los SHAP values para cada feature (global importance)
importance = np.abs(shap_values_class_1).mean(axis=0)
feature_names = X_train.columns

# Guardar como CSV para uso en Streamlit
df_shap_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': importance
}).sort_values(by='importance', ascending=False)

df_shap_importance.to_csv("data/shap_global_importance.csv", index=False)


# Prueba de diferentes modelos

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier
import lazypredict.Supervised
from tqdm import tqdm

# Forzar barra CLI en lugar de notebook
lazypredict.Supervised.notebook_tqdm = tqdm
lazypredict.Supervised.use_notebook_tqdm = False

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, stratify=y_resampled, test_size=0.2, random_state=42)

# Comparador de modelos
clf = LazyClassifier(verbose=1, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# Mostrar top 5 por ROC AUC
print(models.sort_values(by="ROC AUC", ascending=False).head())

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.9996956786366403, 'Balanced Accuracy': np.float64(0.9996956786366402), 'ROC AUC': np.float64(0.9996956786366402), 'F1 Score': 0.9996956786084568, 'Time taken': 0.4336071014404297}
{'Model': 'BaggingClassifier', 'Accuracy': 0.9996956786366403, 'Balanced Accuracy': np.float64(0.9996956786366402), 'ROC AUC': np.float64(0.9996956786366402), 'F1 Score': 0.9996956786084568, 'Time taken': 0.17564797401428223}
{'Model': 'BernoulliNB', 'Accuracy': 0.7486305538648813, 'Balanced Accuracy': np.float64(0.7486305538648813), 'ROC AUC': np.float64(0.7486305538648813), 'F1 Score': 0.7368947360258079, 'Time taken': 0.015197992324829102}


{'Model': 'CalibratedClassifierCV', 'Accuracy': 0.95830797321972, 'Balanced Accuracy': np.float64(0.95830797321972), 'ROC AUC': np.float64(0.9583079732197202), 'F1 Score': 0.9582550506963626, 'Time taken': 0.2080059051513672}
{'Model': 'DecisionTreeClassifier', 'Accuracy': 0.9996956786366403, 'Balanced Accuracy': np.float64(0.9996956786366402), 'ROC AUC': np.float64(0.9996956786366402), 'F1 Score': 0.9996956786084568, 'Time taken': 0.03367900848388672}
{'Model': 'DummyClassifier', 'Accuracy': 0.5, 'Balanced Accuracy': np.float64(0.5), 'ROC AUC': np.float64(0.5), 'F1 Score': 0.3333333333333333, 'Time taken': 0.009619951248168945}
{'Model': 'ExtraTreeClassifier', 'Accuracy': 0.9835666463785758, 'Balanced Accuracy': np.float64(0.9835666463785757), 'ROC AUC': np.float64(0.9835666463785758), 'F1 Score': 0.9835665915893667, 'Time taken': 0.01503896713256836}


{'Model': 'ExtraTreesClassifier', 'Accuracy': 0.9942178940961656, 'Balanced Accuracy': np.float64(0.9942178940961656), 'ROC AUC': np.float64(0.9942178940961657), 'F1 Score': 0.9942178892767565, 'Time taken': 0.3470931053161621}
{'Model': 'GaussianNB', 'Accuracy': 0.9321363359707852, 'Balanced Accuracy': np.float64(0.9321363359707852), 'ROC AUC': np.float64(0.9321363359707853), 'F1 Score': 0.931906277253291, 'Time taken': 0.011677265167236328}
{'Model': 'KNeighborsClassifier', 'Accuracy': 0.9640900791235545, 'Balanced Accuracy': np.float64(0.9640900791235545), 'ROC AUC': np.float64(0.9640900791235544), 'F1 Score': 0.964066597842224, 'Time taken': 0.10783219337463379}
{'Model': 'LabelPropagation', 'Accuracy': 0.8904443091905052, 'Balanced Accuracy': np.float64(0.8904443091905052), 'ROC AUC': np.float64(0.8904443091905052), 'F1 Score': 0.8896198052838725, 'Time taken': 1.8785762786865234}


{'Model': 'LabelSpreading', 'Accuracy': 0.888618381010347, 'Balanced Accuracy': np.float64(0.888618381010347), 'ROC AUC': np.float64(0.8886183810103471), 'F1 Score': 0.8877561724604315, 'Time taken': 2.985320806503296}
{'Model': 'LinearDiscriminantAnalysis', 'Accuracy': 0.7997565429093122, 'Balanced Accuracy': np.float64(0.7997565429093122), 'ROC AUC': np.float64(0.7997565429093122), 'F1 Score': 0.7992191585841298, 'Time taken': 0.0441899299621582}
{'Model': 'LinearSVC', 'Accuracy': 0.9528301886792453, 'Balanced Accuracy': np.float64(0.9528301886792452), 'ROC AUC': np.float64(0.9528301886792452), 'F1 Score': 0.9527277050657266, 'Time taken': 0.0846109390258789}
{'Model': 'LogisticRegression', 'Accuracy': 0.9400486914181375, 'Balanced Accuracy': np.float64(0.9400486914181376), 'ROC AUC': np.float64(0.9400486914181376), 'F1 Score': 0.9398662428872735, 'Time taken': 0.037725210189819336}
{'Model': 'NearestCentroid', 'Accuracy': 0.8037127206329885, 'Balanced Accuracy': np.float64(0.8037127

{'Model': 'NuSVC', 'Accuracy': 0.857273280584297, 'Balanced Accuracy': np.float64(0.8572732805842971), 'ROC AUC': np.float64(0.857273280584297), 'F1 Score': 0.8558280489692977, 'Time taken': 4.0451531410217285}
{'Model': 'PassiveAggressiveClassifier', 'Accuracy': 0.9446135118685332, 'Balanced Accuracy': np.float64(0.9446135118685332), 'ROC AUC': np.float64(0.9446135118685332), 'F1 Score': 0.9444947475064717, 'Time taken': 0.048752784729003906}
{'Model': 'Perceptron', 'Accuracy': 0.9260499087035909, 'Balanced Accuracy': np.float64(0.9260499087035909), 'ROC AUC': np.float64(0.9260499087035909), 'F1 Score': 0.9260244161727615, 'Time taken': 0.01811695098876953}
{'Model': 'QuadraticDiscriminantAnalysis', 'Accuracy': 0.9324406573341448, 'Balanced Accuracy': np.float64(0.9324406573341448), 'ROC AUC': np.float64(0.9324406573341449), 'F1 Score': 0.932426238655913, 'Time taken': 0.015197038650512695}


{'Model': 'RandomForestClassifier', 'Accuracy': 0.9996956786366403, 'Balanced Accuracy': np.float64(0.9996956786366402), 'ROC AUC': np.float64(0.9996956786366402), 'F1 Score': 0.9996956786084568, 'Time taken': 0.7234041690826416}
{'Model': 'RidgeClassifier', 'Accuracy': 0.7997565429093122, 'Balanced Accuracy': np.float64(0.7997565429093122), 'ROC AUC': np.float64(0.7997565429093122), 'F1 Score': 0.7992191585841298, 'Time taken': 0.02030634880065918}
{'Model': 'RidgeClassifierCV', 'Accuracy': 0.7988435788192331, 'Balanced Accuracy': np.float64(0.7988435788192332), 'ROC AUC': np.float64(0.7988435788192331), 'F1 Score': 0.7983100935210345, 'Time taken': 0.025269031524658203}
{'Model': 'SGDClassifier', 'Accuracy': 0.9354838709677419, 'Balanced Accuracy': np.float64(0.935483870967742), 'ROC AUC': np.float64(0.9354838709677419), 'F1 Score': 0.9352192957658464, 'Time taken': 0.03147006034851074}


{'Model': 'SVC', 'Accuracy': 0.9242239805234328, 'Balanced Accuracy': np.float64(0.9242239805234327), 'ROC AUC': np.float64(0.9242239805234328), 'F1 Score': 0.923892075190962, 'Time taken': 1.4781718254089355}
{'Model': 'XGBClassifier', 'Accuracy': 0.9993913572732805, 'Balanced Accuracy': np.float64(0.9993913572732805), 'ROC AUC': np.float64(0.9993913572732804), 'F1 Score': 0.9993913572732805, 'Time taken': 0.12514615058898926}
[LightGBM] [Info] Number of positive: 6570, number of negative: 6570
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000667 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2301
[LightGBM] [Info] Number of data points in the train set: 13140, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gai

100%|██████████| 32/32 [00:13<00:00,  2.42it/s]

{'Model': 'LGBMClassifier', 'Accuracy': 0.9990870359099209, 'Balanced Accuracy': np.float64(0.9990870359099209), 'ROC AUC': np.float64(0.9990870359099209), 'F1 Score': 0.99908703582537, 'Time taken': 0.22999882698059082}
                        Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                    
AdaBoostClassifier          1.00               1.00     1.00      1.00   
BaggingClassifier           1.00               1.00     1.00      1.00   
RandomForestClassifier      1.00               1.00     1.00      1.00   
DecisionTreeClassifier      1.00               1.00     1.00      1.00   
XGBClassifier               1.00               1.00     1.00      1.00   

                        Time Taken  
Model                               
AdaBoostClassifier            0.43  
BaggingClassifier             0.18  
RandomForestClassifier        0.72  
DecisionTreeClassifier        0.03  
XGBClassifier                 0.13  


## ✅ Conclusión de Selección de Modelo

### 🔍 Observaciones:

- Todos los modelos top (`AdaBoost`, `Bagging`, `RandomForest`, `DecisionTree`, `XGBoost`) alcanzaron métricas sobresalientes:
  - **Accuracy**, **Balanced Accuracy**, **ROC AUC** y **F1 Score** ≈ **1.00**
  - Tiempos de entrenamiento muy bajos (< 1 segundo)
- Esto sugiere que el conjunto de datos preprocesado y balanceado es **altamente separable**, lo cual es común en datasets simulados o bien estructurados. Aun que en proyectos reales rara vez nos encontramos con resultados así

---

## ✅ Conclusión sobre el Mejor Modelo

### 🔍 ¿Qué encontramos?

Probamos varios modelos de predicción y todos obtuvieron resultados excelentes, con una precisión cercana al 100%. Esto indica que los datos con los que entrenamos el modelo permiten distinguir muy bien entre transacciones normales y fraudulentas.

---

### 🧠 Comparación de Modelos

| Modelo                 | ¿Qué lo hace bueno?                                              | ¿Qué tener en cuenta?                                      |
|------------------------|------------------------------------------------------------------|-------------------------------------------------------------|
| **Random Forest**      | Muy confiable, funciona bien con datos variados y poco limpios  | Un poco más difícil de entender cómo toma decisiones        |
| **XGBoost**            | Muy potente y preciso, ideal si los datos son complejos          | Más técnico y requiere ajustes finos                        |
| **Árbol de decisión**  | Muy fácil de entender, rápido                                    | Puede equivocarse si los datos cambian mucho                |
| **AdaBoost**           | Combina varios modelos para hacerlo más fuerte                   | Puede fallar si hay datos muy raros o extremos              |
| **Bagging**            | Hace muchas versiones del mismo modelo y vota el resultado final | Puede usar más recursos y ser más lento                     |

---

### ✅ Nuestra Elección Final: `RandomForestClassifier`

Se eligió **Random Forest** como el mejor modelo porque:

- Tiene un rendimiento excelente en la predicción de fraudes.
- Funciona bien incluso si los datos no son perfectos.
- Es rápido de entrenar.
- Se puede usar en producción y es compatible con herramientas para entender sus decisiones.

---